In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install langchain_upstage
#!pip install -U langchain-community
#!pip install -q semantic_version predibase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
import os



from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader , TextLoader, PyMuPDFLoader
from langchain_community.llms import OpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Define the PDF loader
from IPython.display import Markdown

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader

file_path = "/content/sample_data/journal.pone.0276781.pdf" # 링크는 안됨

# For image files, set use_ocr to True to perform OCR inference on the document before layout detection.
loader = UpstageLayoutAnalysisLoader(file_path, split="page", api_key=UPSTAGE_API_KEY, use_ocr=True)

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
pages = loader.load()  # or loader.lazy_load()

for page in pages:
    print(page) # print the document content

page_content='<figure><img id='0' style='font-size:18px' alt="a1111111111
a1111111111
a1111111111
a1111111111
a1111111111" data-coord="top-left:(75,504); bottom-right:(188,618)" /></figure> <p id='1' data-category='paragraph' style='font-size:14px'>OPEN ACCESS</p> <br><p id='2' data-category='paragraph' style='font-size:14px'>Citation: Pavey H, Kulkarni S, Wood A, Ben-<br>Shlomo Y, Sever P, McEniery C, et al. (2022)<br>Primary hypertension, anti-hypertensive<br>medications and the risk of severe COVID-19 in UK<br>Biobank. PLoS ONE 17(11): e0276781. https://doi.<br>org/10.1371/journal.pone.0276781</p> <br><p id='3' data-category='paragraph' style='font-size:14px'>Editor: Sadashiva S. Karnik, Cleveland Clinic Lerner<br>Research Institute, UNITED STATES</p> <p id='4' data-category='paragraph' style='font-size:14px'>Received: March 22, 2022</p> <p id='5' data-category='paragraph' style='font-size:14px'>Accepted: October 13, 2022</p> <br><p id='6' data-category='paragraph' style='font-size:

In [ ]:
# pages 내에 68페이지에 해당하는 OCR결과물이 저장됨
pages[0]
# metadata, page_content로 구성된 개별 결과물임.

Document(metadata={'page': 1}, page_content='<figure><img id=\'0\' style=\'font-size:18px\' alt="a1111111111\na1111111111\na1111111111\na1111111111\na1111111111" data-coord="top-left:(75,504); bottom-right:(188,618)" /></figure> <p id=\'1\' data-category=\'paragraph\' style=\'font-size:14px\'>OPEN ACCESS</p> <br><p id=\'2\' data-category=\'paragraph\' style=\'font-size:14px\'>Citation: Pavey H, Kulkarni S, Wood A, Ben-<br>Shlomo Y, Sever P, McEniery C, et al. (2022)<br>Primary hypertension, anti-hypertensive<br>medications and the risk of severe COVID-19 in UK<br>Biobank. PLoS ONE 17(11): e0276781. https://doi.<br>org/10.1371/journal.pone.0276781</p> <br><p id=\'3\' data-category=\'paragraph\' style=\'font-size:14px\'>Editor: Sadashiva S. Karnik, Cleveland Clinic Lerner<br>Research Institute, UNITED STATES</p> <p id=\'4\' data-category=\'paragraph\' style=\'font-size:14px\'>Received: March 22, 2022</p> <p id=\'5\' data-category=\'paragraph\' style=\'font-size:14px\'>Accepted: October 1

In [ ]:
# OCR 결과물을 json으로 저장
import json

pages_list = [
    {"text": page.page_content, "metadata": page.metadata} for page in pages
]

with open("ocr_result.json", "w", encoding="utf-8") as file:
    json.dump(pages_list, file, ensure_ascii=False, indent=4)

In [ ]:
# JSON 파일에서 Document 리스트 불러오기
def load_documents_from_json(file_name):

    from langchain_core.documents.base import Document

    with open(file_name, "r", encoding="utf-8") as file:
        docs_list = json.load(file)
    return [
        Document(page_content=doc["text"], metadata=doc["metadata"]) for doc in docs_list
    ]

In [ ]:
# 불러오기
load_pages = load_documents_from_json("./ocr_result.json")
for page in load_pages:
    print(page)

page_content='<figure><img id='0' style='font-size:18px' alt="a1111111111
a1111111111
a1111111111
a1111111111
a1111111111" data-coord="top-left:(75,504); bottom-right:(188,618)" /></figure> <p id='1' data-category='paragraph' style='font-size:14px'>OPEN ACCESS</p> <br><p id='2' data-category='paragraph' style='font-size:14px'>Citation: Pavey H, Kulkarni S, Wood A, Ben-<br>Shlomo Y, Sever P, McEniery C, et al. (2022)<br>Primary hypertension, anti-hypertensive<br>medications and the risk of severe COVID-19 in UK<br>Biobank. PLoS ONE 17(11): e0276781. https://doi.<br>org/10.1371/journal.pone.0276781</p> <br><p id='3' data-category='paragraph' style='font-size:14px'>Editor: Sadashiva S. Karnik, Cleveland Clinic Lerner<br>Research Institute, UNITED STATES</p> <p id='4' data-category='paragraph' style='font-size:14px'>Received: March 22, 2022</p> <p id='5' data-category='paragraph' style='font-size:14px'>Accepted: October 13, 2022</p> <br><p id='6' data-category='paragraph' style='font-size:

Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Use the recursive character splitter
recur_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=60,
    separators=["\n\n", "\n", "\.", " ", ""]
)

# Perform the splits using the splitter
data_splits = recur_splitter.split_documents(pages)
print(data_splits)
print("Number of splits:", len(data_splits))

# Print a random chunk
import random
content = random.choice(data_splits).page_content
print("Content length:", len(content))
print(content)

[Document(metadata={'page': 1}, page_content='<figure><img id=\'0\' style=\'font-size:18px\' alt="a1111111111\na1111111111\na1111111111\na1111111111'), Document(metadata={'page': 1}, page_content='a1111111111" data-coord="top-left:(75,504); bottom-right:(188,618)" /></figure> <p id=\'1\' data-category=\'paragraph\' style=\'font-size:14px\'>OPEN ACCESS</p> <br><p id=\'2\' data-category=\'paragraph\' style=\'font-size:14px\'>Citation: Pavey H, Kulkarni S, Wood A, Ben-<br>Shlomo Y, Sever P, McEniery C, et al. (2022)<br>Primary hypertension, anti-hypertensive<br>medications and the risk of severe COVID-19 in UK<br>Biobank. PLoS ONE 17(11): e0276781. https://doi.<br>org/10.1371/journal.pone.0276781</p> <br><p id=\'3\' data-category=\'paragraph\' style=\'font-size:14px\'>Editor: Sadashiva S. Karnik, Cleveland Clinic Lerner<br>Research Institute, UNITED STATES</p> <p id=\'4\' data-category=\'paragraph\' style=\'font-size:14px\'>Received: March 22, 2022</p> <p id=\'5\' data-category=\'paragrap

In [ ]:
print(len(data_splits)) # After chunking

66


Vector Store 구축

In [ ]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(
  api_key=UPSTAGE_API_KEY,
  model="solar-embedding-1-large"
)

# Test
doc_result = embeddings.embed_documents(
    ["SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling.", "DUS is simple yet effective in scaling up high performance LLMs from small ones."]
)

query_result = embeddings.embed_query("What makes Solar LLM small yet effective?")
print(query_result)

[0.01824951171875, -0.0130157470703125, -0.0015535354614257812, 0.00690460205078125, 0.0174560546875, 0.0005154609680175781, -0.0177154541015625, 0.0178680419921875, -0.004154205322265625, 0.0433349609375, 0.0029544830322265625, 0.031707763671875, -0.00238037109375, -0.0232391357421875, 0.011962890625, -0.00249481201171875, 0.0040283203125, 0.016326904296875, -0.04058837890625, -0.0019664764404296875, 0.009552001953125, 0.0028533935546875, 0.0158843994140625, 0.003986358642578125, -0.0020923614501953125, -0.007328033447265625, 0.0036716461181640625, -0.01010894775390625, 0.036163330078125, -0.0018243789672851562, 0.027313232421875, 0.00299835205078125, 0.007289886474609375, 0.006221771240234375, 0.01114654541015625, -0.004283905029296875, -0.01129150390625, 0.021697998046875, -0.0004239082336425781, -0.0255889892578125, 0.01416015625, -0.0017213821411132812, -0.02081298828125, 0.000629425048828125, 0.016876220703125, -0.005489349365234375, 0.0192413330078125, 0.0037059783935546875, 0.0

In [ ]:
print(len(doc_result))
print(doc_result)

2
[[-0.0072784423828125, -0.01180267333984375, -0.019317626953125, 0.0164794921875, 0.01085662841796875, 0.0018177032470703125, 0.0013208389282226562, 0.005397796630859375, -0.0096893310546875, 0.0098876953125, -0.01027679443359375, 0.019561767578125, -0.007598876953125, 0.01425933837890625, 0.004795074462890625, 0.005161285400390625, -0.01580810546875, 0.006549835205078125, -0.0093231201171875, -0.00800323486328125, -0.007083892822265625, -0.0193634033203125, -0.004848480224609375, -0.006336212158203125, 0.012786865234375, -0.0052642822265625, 0.0230712890625, -0.01568603515625, 0.014556884765625, 0.01959228515625, -0.00045180320739746094, 0.01047515869140625, 0.01129150390625, -0.0223236083984375, -0.002368927001953125, -0.00374603271484375, -0.017547607421875, 0.00830841064453125, 0.005893707275390625, -0.0239715576171875, -0.0035076141357421875, 0.007686614990234375, -0.0081634521484375, 0.0017671585083007812, -0.0020122528076171875, 0.01534271240234375, 0.0272674560546875, -0.0061

ChromaDB 생성

In [ ]:
# 1. embedding 처리, 2. db폴더에 데이터 저장
persist_directory = 'paper_db'

vectordb = Chroma.from_documents(
    documents=data_splits, # 위에서 처리한 데이터
    embedding=embeddings, # upstage solar embedding 1 large
    persist_directory=persist_directory)

In [ ]:
# 2.DB 초기화
vectordb.persist()
vectordb = None

In [ ]:
# DB 불러오기
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(
  api_key=UPSTAGE_API_KEY,
  model="solar-embedding-1-large"
)

persist_directory = 'paper_db'

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("covid-19")

for doc in docs:
    print(doc.metadata) # covid-19

<ipython-input-31-8f30ef0b87f9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents("covid-19")


{'page': 13}
{'page': 13}
{'page': 13}
{'page': 13}


Make chain Test

In [ ]:
from langchain_community.llms import Predibase

# Base
model = Predibase(
    model="solar-1-mini-chat-240612",
    predibase_api_key=PREDIBASE_API_KEY,
    predibase_sdk_version=None,  # optional parameter (defaults to the latest Predibase SDK version if omitted)
)

# 만약 Adapter fine-tuning 쓰려면 ?
# # The fine-tuned adapter is hosted at Predibase (adapter_version must be specified).
# model = Predibase(
#     model="solar-1-mini-chat-240612",
#     predibase_api_key=api_token,
#     predibase_sdk_version=None,  # optional parameter (defaults to the latest Predibase SDK version if omitted)
#     adapter_id="e2e_nlg",
#     adapter_version=1,
# )


response = model.invoke("Tell me what type of data is used in this sentence.")
print(response)

Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)



Sure, I can recommend a few nice dry wines for you. Here are a few options:

1. Pinot Noir: This is a light-bodied red wine that is known for its delicate flavors and aromas. It has a dry finish and is a great choice for those who prefer a lighter red wine.
2. Sauvignon Blanc: This is a white wine that is known for its crisp, refreshing flavors. It has a dry finish and is a great choice for those who prefer a lighter, more acidic white wine.
3. Chardonnay: This is a white wine that can range from light and crisp to full-bodied and oaky. If you prefer a dry, crisp Chardonnay, look for one that is made in a stainless steel tank or one that is made in a cooler climate.
4. Riesling: This is a white wine that can range from sweet to dry. If you prefer a dry Riesling, look for one that is made in a cooler climate or one that is labeled as "Kabinett" or "Trocken."

I hope these recommendations help!


Make chain

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff", # 이 체인 유형은 주어진 모든 문서의 내용을 하나로 합치고 하나의 큰 텍스트로 전체를 llm에 전달
    # "map_reduce, refine, map_rerank" 등의 유형이 있으며, 가장 적절한 것을 선택해야 함.

    retriever=retriever,
    return_source_documents=True)

def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata)

Test

In [ ]:
query = "Tell me what type of data is used in this sentence"
llm_response = qa_chain(query)
process_llm_response(llm_response)

<ipython-input-38-e719c689af49>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  llm_response = qa_chain(query)


Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)

The UK Biobank data that support the findings of this study are available from the UK Biobank (www.ukbiobank.ac.uk), subject to approval by UK Biobank.


Sources:
{'page': 1}
{'page': 1}
{'page': 2}
{'page': 2}


이 연구 결과를 뒷받침하는 영국 바이오뱅크 데이터는 영국 바이오뱅크(www.ukbiobank.ac.uk)에서 제공되며, 영국 바이오뱅크의 승인을 받아야 합니다.

In [ ]:
query = "Please provide detailed output of the categories and names of data used, based on classification, such as basic health data, hypertension-related data, primary care records, socioeconomic data, COVID-19-related data, inflammation data, Hospital Episode Statistics (HES) data, and self-reported data."
llm_response = qa_chain(query)
process_llm_response(llm_response)

Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)



The categories and names of data used in the analyses are as follows:

1. Basic health data: UK Biobank cohort data, including self-reported data, hospital episodes from Hospital Episode Statistics (HES) data, and primary health care records for the individuals, where available.
2. Hypertension-related data: ICD10 codes for essential (primary) hypertension (I10), obtained from the UK Biobank cohort data.
3. Primary care records: Read coded clinical terms (version 2 (V2) and clinical terms version 3 (CTV3), Systematised Nomenclature of Medicine–Clinical Terms (SNOMED—CT), The Phoenix Partnership (TPP) and TPP local codes via SystmOne and Egton Medical Information Systems (EMIS) local codes were mapped to ICD10 codes for linked health records where ICD10 codes were not provided.
4. Socioeconomic data: Not explicitly mentioned in the provided context.
5. COVID-19-related data: COVID-19 test results from SARS-COV2 polymerase chain reaction


Sources:
{'page': 4}
{'page': 4}
{'page': 3}
{

1. 기본 건강 데이터: 자체 보고 데이터, 병원 에피소드 통계(HES) 데이터의 병원 에피소드, 해당 개인의 기본 의료 기록(가능한 경우)을 포함한 UK Biobank 코호트 데이터.
2. 고혈압 관련 데이터: UK Biobank 코호트 데이터에서 얻은 필수(1차) 고혈압(I10)에 대한 ICD10 코드.
3. 1차 진료 기록: 코드화된 임상 용어(버전 2(V2) 및 임상 용어 버전 3(CTV3), Systematised Nomenclature of Medicine–Clinical Terms(SNOMED—CT), The Phoenix Partnership(TPP) 및 SystmOne 및 Egton Medical Information Systems(EMIS) 로컬 코드를 통한 TPP 로컬 코드를 읽고 ICD10 코드가 제공되지 않은 연결된 건강 기록의 경우 ICD10 코드에 매핑했습니다.
4. 사회경제적 데이터: 제공된 맥락에서 명시적으로 언급되지 않음.
5. COVID-19 관련 데이터: SARS-COV2 중합효소 연쇄 반응에서 얻은 COVID-19 검사 결과

In [ ]:
query = "Please print out the categories and names of the data used in the experiment in detail."
llm_response = qa_chain(query)
process_llm_response(llm_response)

Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)

The data used in the experiment includes UK Biobank Resource (45885), COVID-19 test results from SARS-COV2 polymerase chain reaction (PCR)-based swab tests provided by Public Health England (PHE), and Hospital Episode Statistics (HES) data.


Sources:
{'page': 12}
{'page': 12}
{'page': 4}
{'page': 4}


실험에 사용된 데이터에는 영국 바이오뱅크 자원(45885), 영국 공중보건원(PHE)이 제공한 SARS-COV2 중합효소 연쇄 반응(PCR) 기반 면봉 검사를 통한 COVID-19 검사 결과, 병원 에피소드 통계(HES) 데이터가 포함됩니다.

Template 사용

In [ ]:
qa_template = """Here is a command that describes the task. Write a response that properly completes the request. If you cannot give an appropriate answer, say I don't know or I can't find it. Don't try to make an appropriate answer. Give an answer that is as accurate and as supportive as possible.

Context: {context}

Question: {question}
Answer:
"""

from langchain.prompts import PromptTemplate
qa_prompt_template = PromptTemplate.from_template(qa_template)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff", # 이 체인 유형은 주어진 모든 문서의 내용을 하나로 합치고 하나의 큰 텍스트로 전체를 llm에 전달
    # "map_reduce, refine, map_rerank" 등의 유형이 있으며, 가장 적절한 것을 선택해야 함.
    chain_type_kwargs={"prompt":qa_prompt_template},
    retriever=retriever,
    return_source_documents=True)

In [ ]:
queries = ["Tell me what type of data is used in this sentence",
"Please print out the categories and names of the data used in the experiment in detail.",
"Please provide detailed output of the categories and names of data used, based on classification, such as basic health data, hypertension-related data, primary care records, socioeconomic data, COVID-19-related data, inflammation data, Hospital Episode Statistics (HES) data, and self-reported data."
]

In [ ]:
for query in queries :
    qa_response = qa_chain({"query": f"{query}"})
    print(qa_response['result'])

Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)

The UK Biobank data that support the findings of this study are available from the UK Biobank (www.ukbiobank.ac.uk), subject to approval by UK Biobank.


Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)


The data used in the experiment includes the following categories and names:

1. UK Biobank Resource (45885)
2. COVID-19 test results from SARS-COV2 polymerase chain reaction (PCR)-based swab tests provided by Public Health England (PHE)
3. Hospital Episode Statistics (HES) data
4. World Health Organisation (WHO) clinical progression scale
5. ICD10 codes: U071 and U072 for COVID-19 related cause of death
6. Supplementary information on specific medication lists and code lists.


Connected to Predibase as User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)


The data used in the study can be classified into the following categories:

1. Basic health data: This includes information on the participants' health status, such as their age, sex, and ethnicity.
2. Hypertension-related data: This includes information on the participants' hypertension status, such as whether they have been diagnosed with hypertension and whether they are taking blood pressure-lowering medication.
3. Primary care records: This includes information on the participants' primary care records, such as their Read coded clinical terms, SNOMED-CT, TPP, and EMIS local codes.
4. Socioeconomic data: This includes information on the participants' socioeconomic status, such as their townsend deprivation score.
5. COVID-19-related data: This includes information on the participants' COVID-19 status, such as whether they have tested positive for COVID-19 and whether they have been hospitalized or have died due to COVID-19.
6. Inflammation data: This includes information on the p

Predibase에 User(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)로 연결됨
이 연구 결과를 뒷받침하는 UK Biobank 데이터는 UK Biobank(www.ukbiobank.ac.uk)에서 제공되며 UK Biobank의 승인을 받아야 합니다.
Predibase에 User로 연결됨(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)

실험에 사용된 데이터에는 다음 범주와 이름이 포함됩니다.

1. UK Biobank Resource(45885)
2. Public Health England(PHE)에서 제공한 SARS-COV2 중합효소 연쇄 반응(PCR) 기반 면봉 검사의 COVID-19 검사 결과
3. Hospital Episode Statistics(HES) 데이터
4. World Health Organisation(WHO) 임상 진행 척도
5. ICD10 코드: COVID-19 관련 사망 원인에 대한 U071 및 U072
6. 특정 약물 목록 및 코드 목록에 대한 추가 정보
Predibase에 User로 연결됨(id=ee1714ec-521d-421d-8e3d-36d366d279c6, username=eunkyung0111@gmail.com)

연구에 사용된 데이터는 다음과 같은 범주로 분류할 수 있습니다.

1. 기본 건강 데이터: 여기에는 나이, 성별, 민족과 같은 참가자의 건강 상태에 대한 정보가 포함됩니다.
2. 고혈압 관련 데이터: 여기에는 고혈압 진단을 받았는지 여부, 혈압 강하제를 복용하고 있는지 여부와 같은 참가자의 고혈압 상태에 대한 정보가 포함됩니다.
3. 1차 진료 기록: 여기에는 Read 코딩된 임상 용어, SNOMED-CT, TPP, EMIS 지역 코드와 같은 참가자의 1차 진료 기록에 대한 정보가 포함됩니다.
4. 사회경제적 데이터: 여기에는 타운센드 박탈 점수와 같은 참가자의 사회경제적 상태에 대한 정보가 포함됩니다.
5. COVID-19 관련 데이터: 여기에는 참가자의 COVID-19 상태, 예를 들어 COVID-19에 대해 양성 반응을 보였는지 여부, COVID-19로 인해 입원했거나 사망했는지 여부에 대한 정보가 포함됩니다.
6. 염증 데이터: 여기에는 참가자의 염증 상태, 예를 들어 C-반응성 단백질(CRP) 수치에 대한 정보가 포함됩니다.
7. 병원 에피소드